In [ ]:
import requests
import pandas as pd
import networkx as nx
import numpy as np

import pandas as pd
import numpy as np
import spotipy
import spotipy.util as util
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

import time
import random

import matplotlib
import matplotlib.pyplot as plt

In [ ]:
ARTIST = "Adele"

### API keys and main parameters

SetList.fm

In [ ]:
API_KEY = "ad30e075-cc83-41c3-8708-dfb5625a2330"

BASE_URL = "https://api.setlist.fm/rest/1.0"
MIN_SETLIST_LEN = 5
NUM_OF_PAGES = 6

Spotify

In [ ]:
client_id = '1f9afb3cf4f444d3b3a2f9d3bf20a4dc'
client_secret = '12f145a677b54fe6b6b05c52dcbd433b'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

## SPOTIFY FUNCTIONS

In [ ]:
def get_uri_song_art(song, artist):
    for it in sp.search(song)['tracks']['items']:
        if it['artists'][0]['name'].lower() == artist.lower():
            return it['uri']
    return None

In [ ]:
def get_popularity_song_art(song, artist):
    for it in sp.search(song)['tracks']['items']:
        if it['artists'][0]['name'].lower() == artist.lower():
            return it['popularity'], it['uri']
    return 0, 0

In [ ]:
get_popularity_song_art('hello','adele')

In [ ]:
def get_artist_picture(NAME):
    result = sp.search(NAME) #search query
    #Extract Artist's uri
    artist_uri = [x['uri'] for x in result['tracks']['items'][0]['artists'] if x['name'].lower() == NAME.lower()][0]
    return sp.artist(artist_uri)['images'][0]['url']


In [ ]:
get_artist_picture('The killers')

In [ ]:
def get_albums_info(sp_albums):
    album_names = []
    album_uris = []
    for i in range(len(sp_albums['items'])):
        album_names.append(sp_albums['items'][i]['name'])
        album_uris.append(sp_albums['items'][i]['uri'])
    return album_names, album_uris

In [ ]:
def albumSongs(uri, name, spotify_albums):
    album = uri #assign album uri to a_name
    spotify_albums[album] = {} #Creates dictionary for that specific album
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] #create empty list
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    tracks = sp.album_tracks(album) #pull data on album tracks
    for n in range(len(tracks['items'])): #for each song track
            spotify_albums[album]['album'].append(name) #append album name tracked via album_count
            spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
            spotify_albums[album]['id'].append(tracks['items'][n]['id'])
            spotify_albums[album]['name'].append(tracks['items'][n]['name'])
            spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])
    return spotify_albums

In [ ]:
def get_artist_info(NAME):
    result = sp.search(NAME) #search query
    #Extract Artist's uri
    artist_uri = [x['uri'] for x in result['tracks']['items'][0]['artists'] if x['name'].lower() == NAME.lower()][0]

    #Pull all of the artist's albums
    sp_albums = sp.artist_albums(artist_uri, album_type='album')
    album_names, album_uris = get_albums_info(sp_albums)

    spotify_albums = {}
    album_count = 0
    checked_albs = []
    for i, name in zip(album_uris, album_names): #each album
        if name not in checked_albs:
            checked_albs.append(name)
            spotify_albums = albumSongs(i, name, spotify_albums)
            album_count+=1 #Updates album count once all tracks have been addedb
  
    return spotify_albums

In [ ]:
def audio_features(album):
    #Add new key-values to store audio features
    album['popularity'] = []
    #create a track counter
    track_count = 0
    for track in album['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        pop = sp.track(track)
        album['popularity'].append(pop['popularity'])
        track_count+=1

In [ ]:
def info_pd(spotify_albums):
    sleep_min = 2
    sleep_max = 5
    start_time = time.time()
    request_count = 0


    for i in spotify_albums:
        audio_features(spotify_albums[i])
        request_count+=1
        if request_count % 5 == 0:
            print(str(request_count) + " playlists completed")
            time.sleep(np.random.uniform(sleep_min, sleep_max))
    dic_df = {}
    dic_df['album'] = []
    dic_df['track_number'] = []
    dic_df['id'] = []
    dic_df['name'] = []
    dic_df['uri'] = []
    dic_df['popularity'] = []
    for album in spotify_albums: 
        for feature in dic_df.keys():
            dic_df[feature].extend(spotify_albums[album][feature])

    artist_df = pd.DataFrame.from_dict(dic_df)
    artist_df['name'] = artist_df['name'].apply(lambda x: x.lower())
    return artist_df

## CONCERT INFO FUNCTIONS

In [ ]:
def get_setlist_songs(ARTIST, BASE_URL, API_KEY):
    artist_url = "{}/search/artists/?sort=relevance&artistName={}".format(BASE_URL, ARTIST)
    headers = {'x-api-key': API_KEY, 'Accept': 'application/json'}
    r = requests.get(artist_url, headers=headers)
    artists = r.json().get('artist')
    artist = artists[0]
    responses = []
    for i in range(1, NUM_OF_PAGES): 
        playlist_url = "{}/artist/{}/setlists/?p={}".format(BASE_URL, artist.get('mbid'), i)
        r = requests.get(playlist_url, headers=headers)
        responses.append(r.json())

        
    responses_setlist = map(lambda x: x.get('setlist'), responses)
    flattened_setlists = [item for sublist in responses_setlist for item in sublist if 'tour' in item.keys()]

    cleaned_setlists = list(map(lambda x: {
        'eventDate': x.get('eventDate'), 
        'songs': list(map(lambda z: z.get('name'), \
        [item for sublist in map(lambda y: y.get('song'), x.get('sets').get('set')) for item in sublist])) \
        , 
        'tour': x.get('tour').get('name'),
        'country': x.get('venue').get('city').get('country').get('code')
        }, flattened_setlists))
    
    cleaned_setlists = [x for x in cleaned_setlists if 'songs' in x.keys()]
    cleaned_setlists = [x for x in cleaned_setlists if len(x['songs']) > 5]
    only_songs = list(map(lambda x: ["begin"] + x.get('songs') + ["end"], cleaned_setlists))
    
    return cleaned_setlists, only_songs

In [ ]:
def song_list_to_df(ARTIST, BASE_URL, API_KEY):
    cleaned_list, songs = get_setlist_songs(ARTIST, BASE_URL, API_KEY)
    pairs = []
    for concert in cleaned_list:
        setlist = concert['songs']
        setlist = ['begin'] + setlist + ['end']
        for i in range(len(setlist) - 1):
            pairs.append((setlist[i], setlist[i+1], concert['tour'], concert['country'], concert['eventDate']))

    df_pairs = pd.DataFrame(pairs)

    df_pairs = df_pairs.rename(columns={0:'song_org', 1:'next_song', 2:'tour', 3:'country', 4:'date'})
    the_tour = df_pairs.sort_values(by=['date']).iloc[0]['tour']
    df_pairs2 = df_pairs[df_pairs.tour == the_tour]
    
    length_concert = df_pairs.groupby('date').count().median()['song_org']
    b = df_pairs2.groupby('date').count()
    dates = b[(b.song_org < length_concert + 2) & (b.song_org > length_concert - 2) ].index
    
    df_pairs2 = df_pairs2[df_pairs.date.isin(dates)].groupby(['song_org', 'next_song']).size().reset_index().rename(columns={0:'weight'})
    
    return df_pairs, df_pairs2

### GET CONCERT INFO

In [ ]:
info_complete

In [ ]:
info_complete, df_pairs = song_list_to_df(ARTIST, BASE_URL, API_KEY)

In [ ]:
def get_statistics(info_complete):
    stats = {}
    first_song = info_complete[info_complete.song_org == 'begin'].groupby('next_song').count().\
                sort_values(by=['song_org']).reset_index()
    most_played = info_complete[['song_org', 'next_song']].groupby('next_song').count().\
                sort_values(by='song_org').reset_index()
    
    last_song = info_complete[info_complete.next_song == 'end'].groupby('song_org').count().\
                sort_values(by=['next_song']).reset_index()
    stats['first_song'] = first_song.iloc[-1].next_song
    stats['top_three'] = list(most_played[most_played['next_song'] != 'end'].iloc[-3:]['next_song'].unique()[::-1])
    stats['most_played'] = most_played[most_played['next_song'] != 'end'].iloc[-1]['next_song']
    stats['last_song'] = last_song.iloc[-1].song_org
    
    return stats
    
    

In [ ]:
statistics = get_statistics(info_complete)

In [ ]:
statistics

### GET SPOTIFY INFO

In [ ]:
spotify_albums = get_artist_info(ARTIST)
pd_artist = info_pd(spotify_albums)

In [ ]:
pd_artist

In [ ]:
length_concert = info_complete.groupby('date').count().median()['song_org']

## CREATE GRAPH AND GET PLAYLIST

In [ ]:
G = nx.from_pandas_edgelist(df_pairs, 'song_org', 'next_song', ['weight'], create_using=nx.DiGraph())

In [ ]:
def get_playlist(G, source='begin', target='end'):
    visited = []
    cur = source

    while cur != target:
        visited.append(cur)
        weights = []
        neighs = []
        for n in G.neighbors(cur):
            if n not in visited:
                weights.append(np.exp(G.get_edge_data(cur, n).get('weight')))
                neighs.append(n)
        weights = np.asarray(weights)/sum(weights)
        if len(weights) == 0:
            cur = target
        else:
            cur = np.random.choice(neighs, 1,
                      p=weights)[0]
        
    return visited

In [ ]:
playlist_go = get_playlist(G)

In [ ]:
playlist_go

## VISUALIZE RESULTS

In [ ]:
all_weights = []

f, ax = plt.subplots(figsize=(20,20))
pos=nx.circular_layout(G)
pos_higher = {}
for k, v in pos.items():
    pos_higher[k] = (v[0]*1.1, v[1]*1.1)
    
nx.draw_networkx_nodes(G.nodes, pos=pos)
nx.draw_networkx_labels(G, pos=pos_higher)
colors = ['r', 'b', 'y']
edges_1 = []
edges_2 = []
for i in range(len(playlist_go) - 1):
    edges_1.append((playlist_go[i], playlist_go[i+1]))

for (node1,node2,data) in G.edges(data=True):
    all_weights.append(data['weight'])

unique_weights = list(set(all_weights))

for weight in unique_weights:
    #4 d. Form a filtered list with just the weight you want to draw
    weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in G.edges(data=True) if edge_attr['weight']==weight]
    width = weight*0.1
    nx.draw_networkx_edges(G,pos,edgelist=weighted_edges,width=width, edge_color='k', ax=ax)
    

for ctr, edges_l in enumerate([edges_1]):
    nx.draw_networkx_edges(G,pos=pos, edgelist=edges_l, edge_color = colors[ctr], style='dashed', ax=ax, width=4, alpha=0.5)


#plt.savefig('graph_song_{}'.format(ARTIST),bbox_inches = 'tight',
#    pad_inches = 0)

In [ ]:
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool, ColumnDataSource, LabelSet
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4


In [ ]:
for n in G.nodes:
    

In [ ]:
edge_attrs_width = {}
edge_attrs_color = {}

edges_1 = []
for i in range(len(playlist_go) - 1):
    edges_1.append((playlist_go[i], playlist_go[i+1]))
    
    
for start_node, end_node, data in G.edges(data=True):
    edge_attrs_width[(start_node, end_node)] = data['weight']*0.1
    edge_attrs_color[(start_node, end_node)] = "#ee1010" if (start_node, end_node) in edges_1 else "#CCCCCC"

nx.set_edge_attributes(G, edge_attrs_width, "edge_width")
nx.set_edge_attributes(G, edge_attrs_color, "edge_color")

x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
node_labels = [n for n in G.nodes]
source = ColumnDataSource({'x': np.asarray(x)*1.4 - 0.2, 'y': np.asarray(y)*1.3,
                           'name': [node_labels[i] for i in range(len(x))], 
                          'ft':[5 for i in range(len(x))]})


plot = figure(x_range=(-2,2), y_range=(-2,2), plot_width=800, plot_height=800,
              tools="", toolbar_location=None)

node_hover_tool = HoverTool(tooltips=[("Song", "@name")])
plot.add_tools(node_hover_tool, ResetTool(), TapTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))


graph_renderer.node_renderer.glyph = Circle(size=10, fill_color="#CCCCCC", name='name')
graph_renderer.node_renderer.selection_glyph = Circle(size=10, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=10, fill_color=Spectral4[1], name='name')


graph_renderer.edge_renderer.glyph = MultiLine(line_color="edge_color", line_alpha=0.8, line_width="edge_width")
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width="edge_width")
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width="edge_width")

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()


plot.renderers.append(graph_renderer)
plot.xgrid.visible = False
plot.ygrid.visible = False

plot.xaxis.visible = False
plot.yaxis.visible = False

labels = LabelSet(x='x', y='y', text='name', source=source)

plot.renderers.append(labels)

output_file("interactive_graphs.html")
show(plot)

In [ ]:
#start Bokeh code
edges_1 = []
for i in range(len(playlist_go) - 1):
    edges_1.append((playlist_go[i], playlist_go[i+1]))
    
layout = nx.circular_layout(G)
nodes, nodes_coordinates = zip(*sorted(layout.items()))
nodes_xs, nodes_ys = list(zip(*nodes_coordinates))
nodes_source = ColumnDataSource(dict(x=nodes_xs, y=nodes_ys,name=nodes)) #Can this pass the color? 

hover = HoverTool(tooltips=[('name', '@name')]) #would like to know how to add in more values here manually

plot = figure(plot_width=800, plot_height=800,tools=['tap', hover, 'box_zoom', 'reset'], toolbar_location=None)

r_circles = plot.circle('x', 'y', source=nodes_source, size=10, color='#CCCCCC', level = 'overlay')#this function sets the color of the nodes, but how to set based on the name of the node? 


def get_edges_specs(_network, _layout): 
    d = dict(xs=[], ys=[], alphas=[], colors=[], name=[], width=[])
    weights = [d['weight'] for u, v, d in _network.edges(data=True)]
    max_weight = max(weights)
    calc_alpha = lambda h: 0.1 + 0.6 * (h / max_weight)

    for u, v, data in _network.edges(data=True):
        d['xs'].append([_layout[u][0], _layout[v][0]])
        d['ys'].append([_layout[u][1], _layout[v][1]])
        d['alphas'].append(calc_alpha(data['weight']))
        d['colors'].append(["#ee1010" if (u, v) in edges_1 else "#CCCCCC"])
        d['width'].append(data['weight']*0.1)
        d['name'].append([(u,v)])
    return d

lines_source = ColumnDataSource(get_edges_specs(G, layout))

r_lines = plot.multi_line('xs', 'ys', line_width='width',
                      alpha='alphas', color='colors',
                      source=lines_source)#This function sets the color of the edges

plot.xgrid.visible = False
plot.ygrid.visible = False

plot.xaxis.visible = False
plot.yaxis.visible = False

output_file("interactive_graphs.html")
show(plot)